In [ ]:
!pip install sentence-transformers

In [2]:
import pandas as pd
import time
from tqdm import tqdm
import seaborn as sns
import numpy as np
from textblob import TextBlob
import matplotlib.pyplot as plt
from pathlib import Path
from sentence_transformers import SentenceTransformer

In [23]:
from IPython.display import HTML

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi

Thu Aug 26 07:24:13 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    32W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [10]:
!pip install faiss-gpu

     |████████████████████████████████| 89.7 MB 7.8 kB/s 


In [9]:
model = SentenceTransformer('msmarco-distilbert-base-dot-prod-v3')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.35k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/554 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/341 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/376 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/115 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

### **Pakistan Data**

In [4]:
path = Path('/content/drive/My Drive/PAK-data/ref-summary').resolve()

In [5]:
corpus = list()
def read_text_file(file_path):
    string_without_line_breaks = ""
    with open(file_path, 'r') as f:
        for line in f:
          stripped_line = line.replace("\n"," ")
          string_without_line_breaks += stripped_line
        corpus.append(string_without_line_breaks)
  
import os
# iterate through all file
for file in os.listdir(path):
    # Check whether file is in text format or not
    if file.endswith(".txt"):
        file_path = f"{path}/{file}"
  
        # call read text file function
        read_text_file(file_path)

In [6]:
corpus[:5]

["2018 C L D 177 [Supreme Court of Pakistan] Present: Ejaz Afzal Khan, Qazi Faez Isa and Ijaz ul Ahsan, JJ STATE BANK OF PAKISTAN through Chief Manager, Peshawar and another---Appellants Versus SECURITIES AND EXCHANGE COMMISSION OF PAKISTAN and others---Respondents Civil Appeals Nos. 1406 and 1407 of 2016, decided on 27th October, 2017. (On appeal from the judgment dated 04.02.2016 of the Peshawar High Court, Peshawar passed in I.C.As. Nos. 6 and 7 of 2012) (a) Interpretation of statutes---  ----Words appearing in a section (of a statute) were to be read in the context in which they were used.  In the matter of the Rev. James Godfrey MacManaway PLD 1950 PC 149 ref.  (b) Interpretation of statutes---  ----Reference to dictionary meaning of a word---Propriety---Reading words into an Act unless it was absolutely necessary to do so was contrary to all rules of construction---Reference to dictionaries by the judges, without first examining the statutory provision and its context, was inappr

In [7]:
pak_df = pd.DataFrame(corpus, columns=['Text'])
pak_df.to_csv("Data.csv")

In [11]:
encoded_data_pak = model.encode(corpus)
encoded_data_pak = np.asarray(encoded_data_pak.astype('float32'))

In [16]:
encoded_data_pak

array([[ 4.59445268e-01, -1.31329879e-01, -2.84720153e-01, ...,
        -1.24799833e-01, -7.09990621e-01,  3.47209349e-02],
       [ 1.86555535e-01,  2.83839822e-01, -5.61528683e-01, ...,
        -1.57256976e-01, -7.64637768e-01,  3.06454480e-01],
       [-1.26469046e-01, -4.15720306e-02, -2.99272716e-01, ...,
        -7.93251917e-02, -4.97613519e-01, -1.12997055e-01],
       ...,
       [ 7.97724724e-02, -6.60713762e-02, -6.14597738e-01, ...,
        -2.19592318e-01, -9.57732201e-01,  2.86070630e-02],
       [ 3.71804625e-01, -2.38013804e-01, -4.64141667e-01, ...,
         2.97420025e-01, -7.69170702e-01, -1.08616062e-01],
       [ 1.03992939e-01, -8.73126462e-02, -1.12729430e-01, ...,
         9.56840697e-04, -7.45110393e-01,  2.13671684e-01]], dtype=float32)

In [13]:
import faiss
index_pak = faiss.IndexIDMap(faiss.IndexFlatIP(768))
index_pak.add_with_ids(encoded_data_pak, np.array(range(0, len(pak_df))))
faiss.write_index(index_pak, 'pak.index')

In [22]:
# Serialise index and store it as a pickle
import pickle
with open("faiss_index.pickle", "wb") as h:
    pickle.dump(faiss.serialize_index(index_pak), h)

In [18]:
def fetch_report_info(dataframe_idx):
    info = pak_df.iloc[dataframe_idx]
    meta_dict = {}
    meta_dict['text'] = info['Text']
    return meta_dict
    
def search(query, top_k, index_pak, model):
    t=time.time()
    query_vector = model.encode([query])
    top_k = index_pak.search(query_vector, top_k)
    print('>>>> Results in Total Time: {}'.format(time.time()-t))
    top_k_ids = top_k[1].tolist()[0]
    top_k_ids = list(np.unique(top_k_ids))
    results =  [fetch_report_info(idx) for idx in top_k_ids]
    return results

In [19]:
from pprint import pprint

query="Murder"
results=search(query, top_k=5, index_pak=index_pak, model=model)

print("\n")
for result in results:
    print('\t',result)

>>>> Results in Total Time: 0.01895308494567871


	 {'text': "  2015 S C M R 1406 [Supreme Court of Pakistan] Present: Nasir-ul-Mulk, C.J., Gulzar Ahmed and Mushir Alam, JJ NATIONAL LOGISTIC CELL---Appellant versus IRFAN KHAN and others---Respondents Civil Appeal No.1000 of 2006, decided on 30th January, 2015. (Against judgment dated 26-4-2006 of High Court of Sindh at Karachi, passed in HCA No.329 of 2005)     (a) Fatal Accidents Act (XIII of 1855)---     ----S. 1---Suit for compensation---Composite negligence of the defendants---Joint and several liability---Vicarious liability of an organization for rash and negligent driving by a driver of its vehicle---Scope---Deceased and three other persons were killed in a road accident caused by the rash and negligent driving of the defendant, who was driving a trailer owned by a Government organization---At the time of the accident repairing work on one side of road was being carried out by the City District Government Authority, therefore it

# **Semantic Search Engine**

In [24]:
#@title Sementic search
#@markdown Enter a set of words to find matching cases. 'cases' can be modified to different number of matching cases retured.
search_string = "Murder" #@param {type:"string"}
cases = "3" #@param [1, 2, 3]

output ='<p style="font-family:verdana; font-size:110%;"> '
results=search(search_string, top_k=int(cases), index_pak=index_pak, model=model)
for i in range(len(results)):
  output += " <b>"+results[i].get("text")+"</b>"
  output += "</p><hr>"

output = '<h3>Results:</h3>'+output
display(HTML(output))




>>>> Results in Total Time: 0.016124725341796875
